# LangGraph 기반 검색 에이전트

VectorDB와 웹 검색을 결합한 ReAct 방식의 하이브리드 검색 에이전트입니다.

## 주요 기능
1. ReAct 패턴 (Thought-Action-Observation)
2. 검색 결과 평가 및 재시도
3. 하이브리드 검색 (VectorDB + 웹)
4. 대화 기록 유지

In [ ]:
# !pip install langchain-openai langchain-community langgraph chromadb pymupdf tavily-python langgraph-checkpoint-sqlite -q

## 1. 환경 설정

In [ ]:
import os
import getpass

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
    print("Colab Secrets에서 API 키를 불러왔습니다.")

except (ImportError, KeyError):
    openai_key = getpass.getpass("OpenAI API 키: ")
    os.environ["OPENAI_API_KEY"] = openai_key
    tavily_key = getpass.getpass("Tavily API 키: ")
    os.environ["TAVILY_API_KEY"] = tavily_key
    print("API 키가 입력되었습니다.")

In [ ]:
import json
import sqlite3
import subprocess
from pathlib import Path
from typing import TypedDict, Annotated, List

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
import operator

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

## 2. PDF 다운로드 및 VectorDB 검색기 도구와 웹검색 도구 설정

In [ ]:

pdf_path = "d2l-en.pdf"
url = "https://d2l.ai/d2l-en.pdf"

if not Path(pdf_path).exists():
    print("PDF 다운로드 중...")
    !curl -L -o $pdf_path $url
    print("다운로드 완료")
else:
    print(f"PDF 파일 존재: {pdf_path}")


In [ ]:
chroma_db_path = "./chroma_db_d2l"

if Path(chroma_db_path).exists():
    print("기존 DB 로드...")
    vectorstore = Chroma(persist_directory=chroma_db_path, embedding_function=embeddings)
else:
    print("새 DB 생성...")
    loader = PyMuPDFLoader(pdf_path)
    documents = loader.load()[:100]
    print(f"{len(documents)} 페이지 로드")

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(documents)
    print(f"{len(chunks)}개 청크 생성")

    print("임베딩 생성 중...")
    vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory=chroma_db_path)
    print("완료")

In [ ]:
vectordb_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
web_search_tool = TavilySearchResults(max_results=3)

## 3. State 및 노드 정의

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[List, operator.add]
    question: str
    current_query: str
    search_method: str
    search_results: str
    is_relevant: bool
    iteration: int
    final_answer: str

def format_history(messages: List[BaseMessage]) -> str:
    if not messages:
        return ""
    history = "\n이전 대화:\n"
    for msg in messages[-6:]:
        role = "사용자" if isinstance(msg, HumanMessage) else "AI"
        history += f"{role}: {msg.content[:200]}...\n"
    return history

In [ ]:
def thought_node(state: AgentState):
    print("\n[Thought] 검색 전략 수립...")
    
    question = state["question"]
    iteration = state.get("iteration", 0)
    messages = state.get("messages", [])
    
    context = format_history(messages)
    if iteration > 0:
        context += f"\n이전 시도 실패, 다시 시도 필요\n"
    
    prompt = f"{context}\n질문: {question}\n\n검색 방법 선택 (vectordb: 딥러닝 교과서, websearch: 웹)\nJSON으로 답변: {{\"search_method\": \"vectordb/websearch\", \"query\": \"검색어\", \"reasoning\": \"이유\"}}"
    
    response = llm.invoke([SystemMessage(content=prompt)])
    
    try:
        result = json.loads(response.content)
        method = result.get("search_method", "websearch")
        query = result.get("query", question)
        print(f"방법: {method}, 쿼리: {query}")
        return {"search_method": method, "current_query": query, "iteration": iteration + 1}
    except:
        print("파싱 실패, 기본값 사용")
        return {"search_method": "websearch", "current_query": question, "iteration": iteration + 1}

In [ ]:
def action_node(state: AgentState):
    print("\n[Action] 검색 수행...")
    
    method = state["search_method"]
    query = state["current_query"]
    
    if method == "vectordb":
        print(f"  VectorDB: '{query}'")
        docs = vectordb_retriever.invoke(query)
        results = "\n\n".join([doc.page_content for doc in docs])
        print(f"{len(docs)}개 문서 검색")
    else:
        print(f"  웹검색: '{query}'")
        search_results = web_search_tool.invoke(query)
        results = "\n\n".join([f"[{r.get('title')}]\n{r.get('content')}" for r in search_results])
        print("검색 완료")
    
    return {"search_results": results}

In [ ]:
def observation_node(state: AgentState):
    print("\n[Observation] 결과 평가...")
    
    question = state["question"]
    results = state["search_results"]
    iteration = state.get("iteration", 0)
    
    eval_prompt = f"""검색 결과가 질문에 답할 수 있는지 평가:
질문: {question}
검색 결과: {results[:500]}...

JSON으로 답변: {{\"is_relevant\": true/false, \"reason\": \"이유\", \"suggestion\": \"제안\"}}"""
    
    eval_response = llm.invoke([SystemMessage(content=eval_prompt)])
    
    try:
        eval_result = json.loads(eval_response.content)
        is_relevant = eval_result.get("is_relevant", False)
        print(f"평가: {'관련 있음' if is_relevant else '관련 없음'}")
        
        if not is_relevant and iteration < MAX_ITERATIONS:
            print(f"재시도 ({iteration}/{MAX_ITERATIONS})")
            return {"is_relevant": False}
        
        print("답변 생성...")
        messages = state.get("messages", [])
        answer_prompt = f"{format_history(messages)}\n질문: {question}\n검색 결과: {results}\n\n위 결과로 답변하세요."
        response = llm.invoke([SystemMessage(content=answer_prompt)])
        
        return {
            "is_relevant": True,
            "final_answer": response.content,
            "messages": [HumanMessage(content=question), AIMessage(content=response.content)]
        }
    except:
        return {"is_relevant": True, "final_answer": "오류 발생", "messages": []}

## 4. 그래프 구성

In [ ]:
MAX_ITERATIONS = 5
conn = sqlite3.connect(":memory:", check_same_thread=False)
memory = SqliteSaver(conn=conn)

def should_continue(state: AgentState) -> str:
    is_relevant = state.get("is_relevant", False)
    iteration = state.get("iteration", 0)
    return "end" if (is_relevant or iteration >= MAX_ITERATIONS) else "continue"

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("thought", thought_node)
workflow.add_node("action", action_node)
workflow.add_node("observation", observation_node)

workflow.set_entry_point("thought")
workflow.add_edge("thought", "action")
workflow.add_edge("action", "observation")
workflow.add_conditional_edges(
    "observation",
    should_continue,
    {"continue": "thought", "end": END}
)

search_agent = workflow.compile(checkpointer=memory)

In [ ]:
search_agent

## 5. 실행 함수

In [ ]:
def run_agent(question: str, thread_id: str = "default", agent: StateGraph = search_agent):
    print("\n" + "=" * 60)
    print(f"질문: {question}")
    print("=" * 60)
    
    config = {"configurable": {"thread_id": thread_id}}
    
    try:
        current_state = agent.get_state(config)
        existing_messages = current_state.values.get("messages", []) if current_state.values else []
    except:
        existing_messages = []
    
    initial_state = {
        "question": question,
        "current_query": question,
        "messages": existing_messages,
        "search_method": "",
        "search_results": "",
        "is_relevant": False,
        "iteration": 0,
        "final_answer": ""
    }
    
    result = agent.invoke(initial_state, config=config)
    
    print("\n" + "=" * 60)
    print("최종 답변:")
    print("=" * 60)
    print(result["final_answer"])
    print(f"\n총 반복: {result['iteration']}")
    
    return result

## 6. 테스트

In [ ]:
# 테스트 1: VectorDB 검색
result1 = run_agent("경사 하강법(gradient descent)이란 무엇인가요?")

In [ ]:
# 테스트 2: 웹 검색
result2 = run_agent("2025년 노벨상은 누가 수상했나요?")

In [ ]:
# 테스트 3: 대화 컨텍스트 유지
result3 = run_agent("activation function이란?", thread_id="test")
result4 = run_agent("그것의 종류는?", thread_id="test")